### Model Preprocessing

Purpose: generate reduced model that contains only the reactions that are allowed to be knocked out
(i.e., excludes blocked reactions, essential reactions, non-GPRs, transport and peripheral reactions, and boundary reactions). 

NOTE: *for each carbon source, the media composition in this notebook is changed accordingly; the reduced model produced is different for each carbon source variations. Below is an example for MR*

Last Updated: Parsa Nayyara (01.02.24)

In [1]:
import cobra
import straindesign as sd

import cobra.test
import os
from os.path import join

data_dir = cobra.test.data_dir

from __future__ import print_function

from cobra import Model, Reaction, Metabolite

In [3]:
model = cobra.io.read_sbml_model(join(data_dir, modelfilepath))
model.solver = "cplex"

In [4]:
print('%i reactions' % len(model.reactions))
print('%i metabolites' % len(model.metabolites))
print('%i genes' % len(model.genes))

2972 reactions
2191 metabolites
1490 genes


In [5]:
# change bounds according to M9 minimal media in Nogales et al. 2020

cobra.Reaction("EX_ca2_e", lower_bound = -10, upper_bound = 1000.0)
cobra.Reaction("EX_co2_e", lower_bound = -100, upper_bound = 1000.0)
cobra.Reaction("EX_cobalt2_e", lower_bound = -10, upper_bound = 1000.0)
cobra.Reaction("EX_cu2_e", lower_bound = -10, upper_bound = 1000.0)
cobra.Reaction("EX_cl_e", lower_bound = -10, upper_bound = 1000.0)
cobra.Reaction("EX_fe2_e", lower_bound = -10, upper_bound = 1000.0)
cobra.Reaction("EX_glc__D_e", lower_bound = -6.3, upper_bound = 1000.0)
cobra.Reaction("EX_h2o_e", lower_bound = -100, upper_bound = 1000.0)
cobra.Reaction("EX_h_e", lower_bound = -100, upper_bound = 1000.0)
cobra.Reaction("EX_hco3_e", lower_bound = -10, upper_bound = 1000.0)
cobra.Reaction("EX_k_e", lower_bound = -10, upper_bound = 1000.0)
cobra.Reaction("EX_mg2_e", lower_bound = -10, upper_bound = 1000.0)
cobra.Reaction("EX_mn2_e", lower_bound = -10, upper_bound = 1000.0)
cobra.Reaction("EX_mobd_e", lower_bound = -10, upper_bound = 1000.0)
cobra.Reaction("EX_nh4_e", lower_bound = -10, upper_bound = 1000.0)
cobra.Reaction("EX_ni2_e", lower_bound = -10, upper_bound = 1000.0)
cobra.Reaction("EX_o2_e", lower_bound = -30, upper_bound = 1000.0)
cobra.Reaction("EX_pi_e", lower_bound = -10, upper_bound = 1000.0)
cobra.Reaction("EX_sel_e", lower_bound = -10, upper_bound = 1000.0)
cobra.Reaction("EX_so4_e", lower_bound = -10, upper_bound = 1000.0)
cobra.Reaction("EX_tungs_e", lower_bound = -10, upper_bound = 1000.0)
cobra.Reaction("EX_zn2_e", lower_bound = -10, upper_bound = 1000.0)
cobra.Reaction("EX_MR", lower_bound = -1000, upper_bound = 1000.0)

medium = model.medium
medium["EX_acmtsoxin_e"] = 0.0
medium["EX_acpptrn_e"] = 0.0
medium["EX_d2one_e"] = 0.0
medium["EX_d3one_e"] = 0.0
medium["EX_d4one_e"] = 0.0
medium["EX_mtsoxin_e"] = 0.0
medium["EX_n2one_e"] = 0.0
medium["EX_pptrn_e"] = 0.0
medium["EX_und2one_e"] = 0.0
medium["EX_gACT[e]"] = 0.0
medium["EX_nh4_e"] = 10.0
medium["EX_o2_e"] = 30.0

# change carbon source
medium["EX_MR"] = 1000
medium["EX_glc__D_e"] = 0.0

model.medium = medium

model.medium

{'EX_ca2_e': 10.0,
 'EX_cl_e': 10.0,
 'EX_co2_e': 100.0,
 'EX_cobalt2_e': 10.0,
 'EX_cu2_e': 10.0,
 'EX_fe2_e': 10.0,
 'EX_h2o_e': 100.0,
 'EX_h_e': 100.0,
 'EX_hco3_e': 10.0,
 'EX_k_e': 10.0,
 'EX_mg2_e': 10.0,
 'EX_mn2_e': 10.0,
 'EX_mobd_e': 10.0,
 'EX_na1_e': 10.0,
 'EX_nh4_e': 10.0,
 'EX_ni2_e': 10.0,
 'EX_o2_e': 30.0,
 'EX_pi_e': 10.0,
 'EX_sel_e': 10.0,
 'EX_so4_e': 10.0,
 'EX_tungs_e': 10.0,
 'EX_zn2_e': 10.0,
 'EX_MR': 1000}

In [6]:
## find reactions that cannot carry flux (blocked reactions)

blocked_reactions = cobra.flux_analysis.variability.find_blocked_reactions(model, reaction_list=None, zero_cutoff=None, open_exchanges=False, processes=None)

In [7]:
## Find essential genes
## essential genes = genes that when removed, biomass == 0

essentialgenes = cobra.flux_analysis.variability.find_essential_genes(model, threshold=0.2, processes=None)

In [8]:
## Find essential reactions
## essential reactions = reactions that when removed, biomass == 0

essential = cobra.flux_analysis.variability.find_essential_reactions(model, threshold=0.2, processes=None)

In [9]:
# list non-GPR reactions

non_gar = ["13DAMPPtex","15DAPabcpp","1PQQS","25DKGLCNt2rpp","25DKGLCNtex","26DAHtex","2DHGLCNtex","2PQQS","34DHCINMtex","34DHCINMtpp","34DHPHEtex","34DHPHEtpp","3HBCOAHL","3MBt2pp","3MBtex","3OXOADPt_ex_","3PQQS","4HBADH","4HPRODCtex","4HPROLTtex","4PCPpp","5APTNtex","5DOAN","5OXPROabcpp","5OXPROtex","AACTOOR","AATA","ABUTtex","ACACtex","ACALDtex","ACALDtpp","ACOADH1","ACOADH2","ACONIs","ACSERtex","ACTNtex","ACTNtpp","ACt2rpp","ACtex","ADEtex","AEPtpp","AGM4PCPpp","AGMabcpp","AGMtex","AGtex","AGtpp","AKGtex","ALAtex","ALDD31","ALDD3y","ALDD4","ALLTNtex","APCtex","APCtpp","APPLDHr","ARBTNR1","ARBTNR2","ARBTNtex","ARBTNtpp","ASNtex","ASO3tex","ASO4tex","ASPtex","ATPM","BALAtex","BILIVERDtex","BILIVERDtpp","BTDDtex","BTDDtpp","BTNt2ipp","BTNtex","CA2tex","CATECHOLtex","CATECHOLtpp","CD2tex","CD2tpp","CELLex","CHLtex","CHOLS_ex_","CHORt2rpp","CHORtex","CINNMCOAH","CINNMtex","CINNMtpp","CLPNH120pp","CLPNH140pp","CLPNH141pp","CLPNH160pp","CLPNH161pp","CLPNH180pp","CLPNH181pp","CLtex","CMCBTFR1","CMCBTFR2","CMCBTFtex","CMCBTFtpp","CMt2ppi","CMtex","COALCDH","COBALT2tex","CONFRLtex","CONFRLtpp","CREATtex","CRNtex","CRO2tex","CRO2tpp","CRO4tex","CRTNtex","CRTNtpp","CSNtex","CTI1","CU2tex","CU2tpp","CUtex","CUtpp","CYANtex","CYSDS","CYSDabcpp","CYSDtex","CYSTL","CYStex","DALAtex","DAPAT","DAPabcpp","DAPtex","DARGabcpp","DARGtex","DASNabcpp","DASNtex","DGBUTNH","DGUDBUTNabcpp","DGUDBUTNtex","DLEUabcpp","DLEUtex","DMETtex","DMGLYtex","DM_5DRIB","DM_AMOB","DM_C100aPHA","DM_C100pPHA","DM_C101PHA","DM_C120aPHA","DM_C121aPHA","DM_C121d6PHA","DM_C140aPHA","DM_C141aPHA","DM_C141d5PHA","DM_C142PHA","DM_C40aPHA","DM_C40atPHA","DM_C40pPHA","DM_C50aPHA","DM_C50pPHA","DM_C60aPHA","DM_C60atPHA","DM_C60pPHA","DM_C70aPHA","DM_C70pPHA","DM_C80aPHA","DM_C80pPHA","DM_C90aPHA","DM_C90pPHA","DM__acmum6p","DM__ppi50","DM_acgam","DM_doxopa","DM_tripeptide_c_","DNMPPA","DNTPPA","DOPAtex","DOPAtpp","DORNtex","DPHEt2rpp","DPHEtex","DPROabcpp","DPROtex","DSERtex","DTMPK","DTYRt2rpp","DTYRtex","DVALtex","D_LACtex","EAR100y","EAR120y","EAR121y","EAR140y","EAR141y","EAR160y","EAR161y","EAR180y","EAR181y","EAR40y","EAR60y","EAR80y","ECOAH12","ECOAH9ir","ECTOtex","EGMEACPR","EPMEACPR","ETHAtex","ETOHtex","EX_12dgr160_e","EX_12dgr180_e","EX_13dampp_e","EX_15dap_e","EX_1ag160_e","EX_1ag180_e","EX_1ag181d9_e","EX_1ag182d9d12_e","EX_25dkglcn_e","EX_26dap_M_e","EX_2ameph_e","EX_2ddglcn_e","EX_2dhglcn_e","EX_2m35mdntha_e","EX_34dhbz_e","EX_34dhcinm_e","EX_34dhphe_e","EX_35dnta_e","EX_3h4atb_e","EX_3mb_e","EX_3oxoadp_e","EX_4abut_e","EX_4hbz_e","EX_4hpro_DC_e","EX_4hpro_LT_e","EX_5aptn_e","EX_5mcsn_e","EX_5oxpro_e","EX_6atha_e","EX_6hnac_e","EX_D_met_e","EX_LalaLglu_e","EX_Ncbmpts_e","EX_R3hdec4e_e","EX_R_3h4atba_e","EX_R_3h6atha_e","EX_R_3hcmrs7e_e","EX_R_3hdcaa_e","EX_R_3hdd5ea_e","EX_R_3hdd6e_e","EX_R_3hdda_e","EX_R_3hhdca_e","EX_R_3hhpa_e","EX_R_3hhxa_e","EX_R_3hnonaa_e","EX_R_3hocta_e","EX_R_3hpba_e","EX_R_3hpdeca_e","EX_R_3hphpa_e","EX_R_3hphxa_e","EX_R_3hpnona_e","EX_R_3hpocta_e","EX_R_3hppta_e","EX_R_3hpt_e","EX_R_3htd58e_e","EX_R_3htd5e_e","EX_R_3httdca_e","EX_T4hcinnm_e","EX_ac_e","EX_acac_e","EX_acald_e","EX_acon_C_e","EX_acser_e","EX_actn_R_e","EX_ade_e","EX_adn_e","EX_adocbl_e","EX_ag_e","EX_agm_e","EX_akg_e","EX_ala_B_e","EX_ala_D_e","EX_ala_L_e","EX_alaala_e","EX_alahis_e","EX_alaleu_e","EX_alathr_e","EX_alatrp_e","EX_algac_MG_14__e","EX_algac_MG_23__e","EX_algac_MG_32__e","EX_algac_MG_41__e","EX_algac_M__e","EX_alltn_e","EX_apc_e","EX_arbtn_e","EX_arbtn_fe3_e","EX_arg_D_e","EX_arg_L_e","EX_asn_D_e","EX_asn_L_e","EX_aso3_e","EX_aso4_e","EX_asp_L_e","EX_balaala_e","EX_balabala_e","EX_balagly_e","EX_balaleu_e","EX_balamd_e","EX_bhb_e","EX_biliverd_e","EX_btd_RR_e","EX_btn_e","EX_butso3_e","EX_bz_e","EX_carn_e","EX_catechol_e","EX_cbi_e","EX_cbl1_e","EX_cd2_e","EX_cell4_e","EX_cgly_e","EX_chol_e","EX_chols_e","EX_chor_e","EX_cinnm_e","EX_cit_e","EX_cl_e","EX_cm_e","EX_cmcbtt_e","EX_co_e","EX_confrl_e","EX_creat_e","EX_crn_e","EX_cro2_e","EX_cro4_e","EX_crtn_e","EX_csn_e","EX_cu_e","EX_cyan_e","EX_cys_D_e","EX_cys_L_e","EX_dag181d9_e","EX_dag182d9d12_e","EX_dca_e","EX_ddca_e","EX_dgudbutn_e","EX_dmanur_e","EX_dmgly_e","EX_dmso2_e","EX_dopa_e","EX_ecto_L_e","EX_enter_e","EX_etha_e","EX_ethso3_e","EX_etoh_e","EX_fald_e","EX_fcmcbtt_e","EX_fe3_e","EX_fe3dcit_e","EX_fe3mcbtt_e","EX_fe3pyovd_e","EX_fe3pyovd_kt_e","EX_fecrm_e","EX_fecrm_un_e","EX_feenter_e","EX_feoxam_e","EX_feoxam_un_e","EX_fer_e","EX_for_e","EX_fru_e","EX_fum_e","EX_ga_e","EX_galct_D_e","EX_galur_e","EX_glc_e","EX_glcn_e","EX_glcr_e","EX_glcur_e","EX_gln_L_e","EX_glu_L_e","EX_glutar_e","EX_gly_e","EX_glyald_e","EX_glyb_e","EX_glyc_R_e","EX_glyc_e","EX_glyclt_e","EX_glycol_e","EX_glygln_e","EX_glyglu_e","EX_glygly_e","EX_glymet_e","EX_glyphe_e","EX_glyser_e","EX_gua_e","EX_gudac_e","EX_gudptn_e","EX_h2o2_e","EX_h2s_e","EX_hcyst_L_e","EX_hdca_e","EX_hdcea_e","EX_hg2_e","EX_hgentis_e","EX_his_L_e","EX_hisgly_e","EX_hishis_e","EX_hom_L_e","EX_hpa_e","EX_hxa_e","EX_hxan_e","EX_icit_e","EX_ile_L_e","EX_ind3ac_e","EX_indole_e","EX_ins_e","EX_isetac_e","EX_lac_D_e","EX_lac_L_e","EX_leu_D_e","EX_leu_L_e","EX_leuleu_e","EX_lnlc_e","EX_lpspput_e","EX_lys_D_e","EX_lys_L_e","EX_m_xyl_e","EX_mal_D_e","EX_mal_L_e","EX_man_e","EX_manur_e","EX_mcbtt_e","EX_meoh_e","EX_met_L_e","EX_mmet_e","EX_mso3_e","EX_nac_e","EX_no2_e","EX_no3_e","EX_no_e","EX_nona_e","EX_o_xyl_e","EX_oaa_e","EX_ocdca_e","EX_ocdcea_e","EX_octa_e","EX_orn_D_e","EX_orn_e","EX_orot_e","EX_oxa_e","EX_oxoarg_L_e","EX_p_xyl_e","EX_pac_e","EX_pacald_e","EX_pb2_e","EX_pea_e","EX_peamn_e","EX_pentso3_e","EX_phe_D_e","EX_phe_L_e","EX_phedca_e","EX_phehpa_e","EX_phehxa_e","EX_pheme_e","EX_phenona_e","EX_pheocta_e","EX_phept_e","EX_phpyr_e","EX_pnto_R_e","EX_ppa_e","EX_ppi_e","EX_prealg_MG_14__e","EX_prealg_MG_23__e","EX_prealg_MG_32__e","EX_prealg_MG_41__e","EX_prealginate_G__e","EX_pro_D_e","EX_pro_L_e","EX_progly_e","EX_pta_e","EX_ptrc_e","EX_ptsla_e","EX_pydam_e","EX_pyovd_e","EX_pyovd_kt_e","EX_pyr_e","EX_quin_kt_e","EX_rib_D_e","EX_sbo3_e","EX_ser_D_e","EX_ser_L_e","EX_sheme_e","EX_skm_e","EX_so2_e","EX_so3_e","EX_spmd_e","EX_succ_e","EX_sulfac_e","EX_tag160_e","EX_tag180_e","EX_tag181d9_e","EX_tag182d9d12_e","EX_taur_e","EX_tcynt_e","EX_thr_L_e","EX_thym_e","EX_tmanur_e","EX_tnt_e","EX_tol_e","EX_trp_L_e","EX_tsul_e","EX_ttdca_e","EX_ttdcea_e","EX_ttrcyc_e","EX_tyr_D_e","EX_tyr_L_e","EX_ura_e","EX_urea_e","EX_uri_e","EX_vacc_e","EX_val_D_e","EX_val_L_e","EX_vanln_e","EX_vanlt_e","EX_xan_e","FALDtex","FALDtpp","FE2tex","FE3PYOVDDR","FE3PYOVDDR2","FECRMR1","FECRMR2","FECRMUtex","FECRMUtpp","FEENTERR1","FEENTERR2","FEENTERabcpp","FEENTERtex","FEENTERtpp","FEOXAMR1","FEOXAMR2","FEOXAMUtex","FEOXAMUtpp","FERtex","FERtpp","FORtex","FUMtex","G3PEabcpp","G3PGabcpp","GACH","GALCTtex","GLCOAS","GLCRtex","GLGC","GLUTARtex","GLXO3","GLYALDtex","GLYBCOAT","GLYBtex","GLYCAtex","GLYCLTtex","GLYCOLtex","GPTNH","GTHRDabc2pp","GUDACabcpp","GUDACtex","GUDPTNabcpp","GUDPTNtex","H2St1pp","H2Stex","HACD9","HAT40tex","HAT40tpp","HCYStabcpp","HCYStex","HG2tex","HG2tppi","HGENTISt2pp","HGENTIStex","HOMtex","HOMtpp","HYXNtex","Htex","ILEtex","IND3ACtex","INDOLEtex","INDOLEtpp","Ktex","LEUtex","LPSALAT","LPSGTR3_kt","LPSGTR4_kt","LYSRC","L_LACtex","MACCOAT","MALDt2_2pp","MALDtex","MALtex","MANabcpp","MANtex","MCBTFR1","MCBTFR2","MCBTFtex","MCBTFtpp","MCSNtex","METtex","MG2tex","MLDCP1App","MLDCP1Bpp","MLDCP2App","MLDCP2Bpp","MLDCP3App","MLDEP1pp","MLDEP2pp","MMETtex","MNtex","MOBDtex","MSULR","M_XYLtpp","M_Xylt1","NADN","NAtex","NCBMPTSabcpp","NCBMPTStex","NH4tex","NI2tex","NMNDA","NO2tex","NO3tex","NOtex","NOtpp","OAAtex","OARGabcpp","OARGtex","OBDHc","ORNtex","OROTtex","OSKTpp","OXAtpp","OXFOtex","O_XYLtppi","O_Xylt1","PACALDtex","PALDO2","PAPA120","PAPA140","PAPA141","PAPA160","PAPA161","PAPA180","PAPA181","PB2tex","PB2tpp","PDE1","PEALtex","PEAMNtex","PETNT161_kt","PETNT181_kt","PHEMEtiex","PHEtex","PHPYRtex","PHPYRtpp","PLIPA1E160pp","PLIPA1E161pp","PLIPA1E180pp","PLIPA1E181d11pp","PLIPA1G160pp","PLIPA1G161pp","PLIPA1G180pp","PLIPA1G181d11pp","PLIPA2E160pp","PLIPA2E161pp","PLIPA2E180pp","PLIPA2E181d11pp","PLIPA2G160pp","PLIPA2G161pp","PLIPA2G180pp","PLIPA2G181d11pp","PMDPHT","PNTOtex","PPAt4pp","PPAtex","PPRDNDH","PPYRDC","PQQH2tpp","PQQtpp","PTAtex","PTRCtex","PYDAMtex","PYDAMtpp","PYNP2r","PYRt2rpp","PYRtex","P_XYLtppi","P_Xylt1","R1PK","RHA100tex","RHA100tpp","RHA101tex","RHA101tpp","RHA120tex","RHA120tpp","RHA121d6tex","RHA121d6tpp","RHA121tex","RHA121tpp","RHA140tex","RHA140tpp","RHA141d5tex","RHA141d5tpp","RHA141tex","RHA141tpp","RHA142tex","RHA142tpp","RHA160tex","RHA160tpp","RHA40tex","RHA40tpp","RHA50tex","RHA50tpp","RHA60tex","RHA60tpp","RHA70tex","RHA70tpp","RHA80tex","RHA80tpp","RHA90tex","RHA90tpp","RHAP100tex","RHAP100tpp","RHAP40tex","RHAP40tpp","RHAP50tex","RHAP50tpp","RHAP60tex","RHAP60tpp","RHAP70tex","RHAP70tpp","RHAP80tex","RHAP80tpp","RHAP90tex","RHAP90tpp","RHAT40tex","RHAT40tpp","RHAT60tex","RHAT60tpp","RIBtex","SBO3rt2pp","SBO3tex","SCYSSL","SELGTHR","SELGTHR2","SELGTHR3","SELtex","SERtex","SKMtex","SO2tex","SO3tex","SO4tex","SPMDtex","SUCCtex","T4HCINNMtex","T4HCINNMtpp","TCYNTtex","THRtex","THYMtex","TNTtex","TNTtpp","TOLtppi","TRPtex","TSULtex","TTRCYCt2pp","TTRCYCtex","TUNGStex","UDCPPtppi","URAtex","UREAtex","URIt2rpp","VALtex","XANtex","Zn2tex","sink_PHAg","sink_pqqA_kt_c_"]

In [10]:
# list transport n peripheral

#transport_reactions = ["DM_ACT","MRex","MRpp","EX_MR","DMPPDApp","DMPPDEex","EX_DMPPDA_e","EX_AP_e","EX_anl_e","Appp","ANLpp","13DAMPPabcpp","13DAMPPtex","15DAPabcpp","25DKGLCNt2rpp","25DKGLCNtex","26DAHtex","2DHGLCNkt_tpp","2DHGLCNtex","34DHCINMtex","34DHCINMtpp","34DHPHEtex","34DHPHEtpp","3MBt2pp","3MBtex","3OXOADPt_ex_","3OXOADPt_pp_","3_4DHBZ1t_pp_","3_4DHBZt2","4ABUTabcpp","4HBZtex","4HPRODCabcpp","4HPRODCtex","4HPROLTabcpp","4HPROLTtex","5APTNabcpp","5APTNtex","5OXPROabcpp","5OXPROtex","6ATHAtexi","6HNACt1pp","6HNACtex","ABUTt2pp","ABUTtex","ACACt2pp","ACACtex","ACALDtex","ACALDtpp","ACONCt4pp","ACONCtex","ACSERtex","ACSERtpp","ACTNtex","ACTNtpp","ACt2rpp","ACt4pp","ACt5pp","ACtex","ADEt2rpp","ADEtex","ADNtex","ADOCBLabcpp","ADOCBLtonex","AEP2abc","AEPtpp","AGM3Pt2pp","AGM4Pt2pp","AGMabcpp","AGMt2pp","AGMtex","AGtex","AGtpp","AKGt2rpp","AKGtex","ALAALAabcpp","ALAALAtex","ALAGLUabcpp","ALAHISabcpp","ALAHIStex","ALALEUabcpp","ALALEUtex","ALATHRabcpp","ALATHRtex","ALATRPabcpp","ALATRPtex","ALAabcpp","ALAt2pp","ALAt4pp","ALAtex","ALLTNt2rpp","ALLTNtex","APCt5","APCt6","APCtex","APCtpp","ARBt3ipp","ARGORNt7pp","ARGabcpp","ARGt2pp","ARGtex","ASNtex","ASO3tex","ASO4t2rpp","ASO4tex","ASPabcpp","ASPt2pp","ASPtex","BALAALAabcpp","BALAALAtex","BALABALAabcpp","BALABALAtex","BALAGLYabcpp","BALAGLYtex","BALALEUabcpp","BALALEUtex","BALAt2pp","BALAtex","BHBtpp","BILIVERDtex","BILIVERDtpp","BLAMDabcpp","BLAMDtex","BTDDtex","BTDDtpp","BTNt2ipp","BTNtex","BUTSO3abcpp","BUTSO3tex","BZt1pp","BZt2_pp","BZtex","CA2tex","CARNabcpp","CARNtex","CATECHOLtex","CATECHOLtpp","CAt6pp","CBItonex","CBIuabcpp","CBL1abcpp","CBL1tonex","CD2tex","CD2tpp","CELLex","CGLYabcpp","CGLYtex","CHLabcpp","CHLtex","CHOLS_ex_","CHOLSabc","CHOLt4ipp","CHORt2rpp","CHORtex","CINNMtex","CINNMtpp","CITt4pp","CITt_kt","CITtex","CLt3_2pp","CLtex","CMt2ex","CMt2ppi","CMtex","CMtexabc","CMtpp","CO2tex","CO2tpp","COBALT2tex","COBALT2tpp","CONFRLtex","CONFRLtpp","COtex","COtpp","CREATtex","CREATtpp","CRNabcpp","CRNtex","CRO2tex","CRO2tpp","CRO4abcpp","CRO4tex","CRTNtex","CRTNtpp","CSNt2pp","CSNtex","CU2tonex","CU2tpp","CUtex","CUtpp","CYANt2pp","CYANtex","CYSDabcpp","CYSDtex","CYSabcpp","CYStex","CYStpp","DALAabcpp","DALAt2pp","DALAtex","DAPabcpp","DAPtex","DARGabcpp","DARGtex","DASNabcpp","DASNtex","DCAtexi","DDCAtexi","DGUDBUTNabcpp","DGUDBUTNtex","DLEUabcpp","DLEUtex","DLYSabcpp","DLYStex","DMETtex","DMGLYBabcpp","DMGLYtex","DMSO2abcpp","DMSO2tex","DOPAtex","DOPAtpp","DORNabcpp","DORNtex","DPHEt2rpp","DPHEtex","DPROabcpp","DPROtex","DSERt2pp","DSERtex","DTYRt2rpp","DTYRtex","DVALabcpp","DVALtex","D_LACt2pp","D_LACtex","ECTOabcpp","ECTOtex","ETHAt2pp","ETHAtex","ETHSO3abcpp","ETHSO3tex","ETOHtex","ETOHtrpp","FALDep","FALDtex","FALDtpp","FALDtpp2","FBACabcpp","FE3DCITtonex","FERtex","FERtpp","FORt2pp","FORtex","FRUtex","FUMt2_2pp","FUMtex","G3PEabcpp","G3PGabcpp","GALCTt2rpp","GALCTtex","GALURt2rpp","GALURtex","GAt1","GAt2pp","GCALDtabcpp","GLCNt2rpp","GLCNtex","GLCRt2rpp","GLCRtex","GLCURt2rpp","GLCURtex","GLCabcpp","GLCtex","GLNtex","GLUTARt2rpp","GLUTARtex","GLUabcpp","GLUt2pp","GLUt4pp","GLUtex","GLYALDtex","GLYALDtpp","GLYBabcpp","GLYBt2pp","GLYBtex","GLYBti4pp","GLYCAt2rpp","GLYCAtex","GLYCLTt2rpp","GLYCLTt4pp","GLYCLTtex","GLYCOLtex","GLYCtex","GLYCtpp","GLYGLNabcpp","GLYGLNtex","GLYGLUabcpp","GLYGLUtex","GLYGLYabcpp","GLYGLYtex","GLYMETabcpp","GLYMETtex","GLYPHEabcpp","GLYPHEtex","GLYSERabcpp","GLYSERtex","GLYabc2pp","GLYt2pp","GLYt4pp","GLYtex","GTHRDabc2pp","GUAt2pp","GUAtex","GUDACabcpp","GUDACtex","GUDPTNabcpp","GUDPTNtex","H2O2tex","H2Otex","H2Otpp","H2St1pp","H2Stex","HAT40tex","HAT40tpp","HCYStabcpp","HCYStex","HDCAtexi","HDCEAtexi","HG2tex","HG2tppi","HGENTISt2pp","HGENTIStex","HISGLYabcpp","HISGLYtex","HISHISabcpp","HISHIStex","HISabcpp","HISt2rpp","HIStex","HOMt2pp","HOMtex","HOMtpp","HPAtexi","HXANt2pp","HXAtexi","HYXNtex","Htex","ICITt4pp","ICITt_kt","ICITtex","ILEabcpp","ILEt2rpp","ILEtex","IND3ACtex","IND3ACtpp","INDOLEt5","INDOLEt6","INDOLEtex","INDOLEtpp","INStex","ISETACabcpp","ISETACtex","Kabcpp","Kt2pp","Kt3pp","Ktex","LALALGLUtex","LCTSt3ipp","LEULEUabcpp","LEULEUtex","LEUabcpp","LEUt2rpp","LEUtex","LNLCtexi","LYSabcpp","LYStex","L_LACt2rpp","L_LACtex","MALt2_2pp","MALtex","MANabcpp","MANtex","MCSNtex","MCSNtpp","MELIBt3ipp","MEOHtex","MEOHtrpp","METD2abcpp","METabcpp","METtex","MG2tex","MG2tpp","MG2uabcpp","MMETt2pp","MMETtex","MNabcpp","MNtex","MOBDabcpp","MOBDtex","MSO3abcpp","MSO3tex","MXYLt5","MXYLt6","M_XYLtpp","M_Xylt1","NACt1pp","NACtex","NAt3_1p5pp","NAt3_1pp","NAt3_2pp","NAt3pp","NAtex","NCBMPTSabcpp","NCBMPTStex","NH4t3pp","NH4tex","NH4tpp","NI2tex","NI2tpp","NI2uabcpp","NO2t2rpp","NO2tex","NO3t2pp","NO3tabcpp","NO3tex","NONAtexi","NOtex","NOtpp","O2tex","O2tpp","OAAt2_2pp","OAAtex","OARGabcpp","OARGtex","OCDCAtexi","OCDCEAtexi","OCTAtexi","ORNabcpp","ORNtex","OROTt2_2pp","OROTtex","OXAtpp","OXFOtex","OXYLt5","OXYLt6","O_XYLtppi","O_Xylt1","PA120abcpp","PA160abcpp","PA161abcpp","PA180abcpp","PA181abcpp","PACALDtex","PACALtabcpp","PACt3","PACtex","PB2tex","PB2tpp","PE120abcpp","PE140abcpp","PE141abcpp","PE160abcpp","PE161abcpp","PE180abcpp","PE181abcpp","PEAL2tabc","PEALtabc","PEALtex","PEAMNtex","PEAtabcpp","PENTSOtex","PG120abcpp","PG140abcpp","PG141abcpp","PG160abcpp","PG161abcpp","PG180abcpp","PG181abcpp","PGP120abcpp","PGP140abcpp","PGP141abcpp","PGP160abcpp","PGP161abcpp","PGP180abcpp","PGP181abcpp","PHEDCAtexi","PHEHPAtexi","PHEHXAtexi","PHEME2abcpp","PHEMEtonex","PHENONAtexi","PHEOCTAtexi","PHEPTtexi","PHEt2rpp","PHEtex","PHPYRtex","PHPYRtpp","PIt2rpp","PIt7pp","PItex","PIuabcpp","PNTOt4pp","PNTOtex","PPAt4pp","PPAtex","PPItex","PPIuabcpp","PQQH2tpp","PQQtpp","PROGLYabcpp","PROGLYtex","PROabcpp","PROt2pp","PROt4pp","PROtex","PTAtex","PTRCabcpp","PTRCt2pp","PTRCtex","PTSLAtexi","PXYLt5","PXYLt6","PYDAMtex","PYDAMtpp","PYRt2rpp","PYRtex","P_XYLtppi","P_Xylt1","QUINtex","QUINtpp","RHA100tex","RHA100tpp","RHA101tex","RHA101tpp","RHA120tex","RHA120tpp","RHA121d6tex","RHA121d6tpp","RHA121tex","RHA121tpp","RHA140tex","RHA140tpp","RHA141d5tex","RHA141d5tpp","RHA141tex","RHA141tpp","RHA142tex","RHA142tpp","RHA160tex","RHA160tpp","RHA40tex","RHA40tpp","RHA50tex","RHA50tpp","RHA60tex","RHA60tpp","RHA70tex","RHA70tpp","RHA80tex","RHA80tpp","RHA90tex","RHA90tpp","RHAP100tex","RHAP100tpp","RHAP40tex","RHAP40tpp","RHAP50tex","RHAP50tpp","RHAP60tex","RHAP60tpp","RHAP70tex","RHAP70tpp","RHAP80tex","RHAP80tpp","RHAP90tex","RHAP90tpp","RHAT40tex","RHAT40tpp","RHAT60tex","RHAT60tpp","RIBabcpp","RIBtex","SBO3rt2pp","SBO3tex","SELabcpp","SELtex","SERt2rpp","SERt4pp","SERtex","SHEME2abcpp","SHEMEtonex","SKMt2pp","SKMtex","SO2tex","SO2tpp","SO3abcpp","SO3tex","SO4t2pp","SO4t4pp_1","SO4tex","SPMDabcpp","SPMDtex","SUCCt2_2pp","SUCCtex","SULFACabcpp","SULFACtex","SULabcpp","T4HCINNMtex","T4HCINNMtpp","TAURabcpp","TAURtex","TCYNTtex","TCYNTtpp","THRabcpp","THRt2pp","THRt4pp","THRtex","THYMt4pp","THYMtex","TNTtex","TNTtpp","TOLt5","TOLt6","TOLt7","TOLtex","TOLtppi","TRPt2rpp","TRPtex","TSULabcpp","TSULtex","TTDCAtexi","TTDCEAtexi","TTRCYCt2pp","TTRCYCt5","TTRCYCt7pp","TTRCYCtex","TTRCYCtpp","TUNGSabcpp","TUNGStex","TYRt2rpp","TYRtex","UHBZ1t_pp_","URAt2pp","URAtex","UREA2t2pp","UREAtabcpp","UREAtex","UREAtpp","URIt2rpp","URItex","VACCtex","VALabcpp","VALt2rpp","VALtex","VANKpp","VANLNtex","VANLTtex","VNLNpp","XANt2pp","XANtex","ZNabcpp","Zn2tex","pentso3abcpp"]
#cell_envelope = ["4PCP","4PCPpp","A5PISO","ACGAMT","ACPS1","AGM3PA","AGM3PApp","AGM3PH","AGM4PA","AGM4PApp","AGM4PCP","AGM4PCPpp","AGM4PH","AGMH","AGMHE","ALAALAr","AM3PA","AM4PA","AM4PCP","ANHMK","BGLApp","CELASEpp","CELLS","COLIPAabcpp_kt","EDTXS1_kt","EDTXS2_kt","EDTXS5_kt","EDTXS6_kt","G1PACT","GF6PTA","GMHEPAT","GMHEPK","GMHEPPA","GTR1_kt","GTR2_kt","HEPCT","HEPK1_kt","HEPK2_kt","HEPK3_kt","HEPT1_kt","HEPT2_kt","KDOCT2","KDOPP","KDOPS","LIPAH","LPADSS2","LPSALAT","LPSGTR1_kt","LPSGTR2_kt","LPSGTR3_kt","LPSGTR4_kt","MCTP1App","MCTP1Bpp","MCTP2App","MDDCP1pp","MDDCP2pp","MDDCP3pp","MDDCP4pp","MDDCP5pp","MDDEP1pp","MDDEP2pp","MDDEP3pp","MDDEP4pp","MI1PP","MLDCP1App","MLDCP1Bpp","MLDCP2App","MLDCP2Bpp","MLDCP3App","MLDEP1pp","MLDEP2pp","MLTGY1pp","MLTGY2pp","MLTGY3pp","MLTGY4pp","MOAT2_kt","MOAT_kt","MPTG","MPTG2","OSKTabcpp","OSKTpp","PAPPT3","PETNT161_kt","PETNT181_kt","PGAMT","PLPSLpp","PLPSabctex","RHMNT1","RHMNT2","S7PI","TDSK2","U23GAAT2","UAAGDS","UACGAMDH2","UAG2E","UAGAAT2","UAGCVT","UAGDP","UAGPT3","UAMAGS","UAMAS","UAPGR","UDCPDP","UDCPDPpp","UDCPGT","UDCPPtppi","UDPG4E","UGMDDS","UHGADA2","UKQUIVORx","UKQUIVORy","UM3PL","UM4PCP","UM4PL","UQUIVOT","USHD1"]
#glycero_metab = ["2AACLPGT160","2AACLPGT161","2AACLPGT180","2AACLPGT181","2AACLPPEAT160","2AACLPPEAT161","2AACLPPEAT180","2AACLPPEAT181","2AGPE160tipp","2AGPE161tipp","2AGPE180tipp","2AGPE181tipp","2AGPG160tipp","2AGPG161tipp","2AGPG180tipp","2AGPG181tipp","ACPPAT120","ACPPAT140","ACPPAT141","ACPPAT160","ACPPAT161","ACPPAT180","ACPPAT181","AGPAT120","AGPAT140","AGPAT141","AGPAT160","AGPAT161","AGPAT180","AGPAT181","ALCD19","ALPATE160pp","ALPATE161pp","ALPATE180pp","ALPATE181pp","ALPATG160pp","ALPATG161pp","ALPATG180pp","ALPATG181pp","APG3PAT120","APG3PAT140","APG3PAT141","APG3PAT160","APG3PAT161","APG3PAT180","APG3PAT181","CLPNH120pp","CLPNH140pp","CLPNH141pp","CLPNH160pp","CLPNH161pp","CLPNH180pp","CLPNH181pp","CLPNS120pp","CLPNS140pp","CLPNS141pp","CLPNS160pp","CLPNS161pp","CLPNS180pp","CLPNS181pp","DAGK120","DAGK140","DAGK141","DAGK160","DAGK161","DAGK180","DAGK181","DASYN120","DASYN140","DASYN141","DASYN160","DASYN161","DASYN180","DASYN181","G3PAT120","G3PAT140","G3PAT141","G3PAT160","G3PAT161","G3PAT180","G3PAT181","G3PD2","G3PD5","GLYALDDr","GLYK","GPDDA2","GPDDA4","LPLIPAL1E160pp","LPLIPAL1E161pp","LPLIPAL1E180pp","LPLIPAL1E181d11pp","LPLIPAL1G160pp","LPLIPAL1G161pp","LPLIPAL1G180pp","LPLIPAL1G181d11pp","PAPA120","PAPA140","PAPA141","PAPA160","PAPA161","PAPA180","PAPA181","PGPP120","PGPP120pp","PGPP140","PGPP140pp","PGPP141","PGPP141pp","PGPP160","PGPP160pp","PGPP161","PGPP161pp","PGPP180","PGPP180pp","PGPP181","PGPP181pp","PGSA120","PGSA140","PGSA141","PGSA160","PGSA161","PGSA180","PGSA181","PLIPA1E160pp","PLIPA1E161pp","PLIPA1E180pp","PLIPA1E181d11pp","PLIPA1G160pp","PLIPA1G161pp","PLIPA1G180pp","PLIPA1G181d11pp","PLIPA2E160pp","PLIPA2E161pp","PLIPA2E180pp","PLIPA2E181d11pp","PLIPA2G160pp","PLIPA2G161pp","PLIPA2G180pp","PLIPA2G181d11pp","PSD120","PSD140","PSD141","PSD160","PSD161","PSD180","PSD181","PSSA120","PSSA140","PSSA141","PSSA160","PSSA161","PSSA180","PSSA181"]
#tRNA_charging = ["ALATRS","ARGTRS","ASPTRS","CYSTRS","FMETTRS","GLNTRAT","GLNTRS","GLUTRS","GLYTRS","HISTRS","ILETRS","LEUTRS","LYSTRS","METTRS","PHETRS","PROTRS","SELCYSS","SERTRS","SERTRS2","THRTRS","TRPTRS","TYRTRS","VALTRS"]

combined = ["ALATRS","ARGTRS","ASPTRS","CYSTRS","FMETTRS","GLNTRAT","GLNTRS","GLUTRS","GLYTRS","HISTRS","ILETRS","LEUTRS","LYSTRS","METTRS","PHETRS","PROTRS","SELCYSS","SERTRS","SERTRS2","THRTRS","TRPTRS","TYRTRS","VALTRS","2AACLPGT160","2AACLPGT161","2AACLPGT180","2AACLPGT181","2AACLPPEAT160","2AACLPPEAT161","2AACLPPEAT180","2AACLPPEAT181","2AGPE160tipp","2AGPE161tipp","2AGPE180tipp","2AGPE181tipp","2AGPG160tipp","2AGPG161tipp","2AGPG180tipp","2AGPG181tipp","ACPPAT120","ACPPAT140","ACPPAT141","ACPPAT160","ACPPAT161","ACPPAT180","ACPPAT181","AGPAT120","AGPAT140","AGPAT141","AGPAT160","AGPAT161","AGPAT180","AGPAT181","ALCD19","ALPATE160pp","ALPATE161pp","ALPATE180pp","ALPATE181pp","ALPATG160pp","ALPATG161pp","ALPATG180pp","ALPATG181pp","APG3PAT120","APG3PAT140","APG3PAT141","APG3PAT160","APG3PAT161","APG3PAT180","APG3PAT181","CLPNH120pp","CLPNH140pp","CLPNH141pp","CLPNH160pp","CLPNH161pp","CLPNH180pp","CLPNH181pp","CLPNS120pp","CLPNS140pp","CLPNS141pp","CLPNS160pp","CLPNS161pp","CLPNS180pp","CLPNS181pp","DAGK120","DAGK140","DAGK141","DAGK160","DAGK161","DAGK180","DAGK181","DASYN120","DASYN140","DASYN141","DASYN160","DASYN161","DASYN180","DASYN181","G3PAT120","G3PAT140","G3PAT141","G3PAT160","G3PAT161","G3PAT180","G3PAT181","G3PD2","G3PD5","GLYALDDr","GLYK","GPDDA2","GPDDA4","LPLIPAL1E160pp","LPLIPAL1E161pp","LPLIPAL1E180pp","LPLIPAL1E181d11pp","LPLIPAL1G160pp","LPLIPAL1G161pp","LPLIPAL1G180pp","LPLIPAL1G181d11pp","PAPA120","PAPA140","PAPA141","PAPA160","PAPA161","PAPA180","PAPA181","PGPP120","PGPP120pp","PGPP140","PGPP140pp","PGPP141","PGPP141pp","PGPP160","PGPP160pp","PGPP161","PGPP161pp","PGPP180","PGPP180pp","PGPP181","PGPP181pp","PGSA120","PGSA140","PGSA141","PGSA160","PGSA161","PGSA180","PGSA181","PLIPA1E160pp","PLIPA1E161pp","PLIPA1E180pp","PLIPA1E181d11pp","PLIPA1G160pp","PLIPA1G161pp","PLIPA1G180pp","PLIPA1G181d11pp","PLIPA2E160pp","PLIPA2E161pp","PLIPA2E180pp","PLIPA2E181d11pp","PLIPA2G160pp","PLIPA2G161pp","PLIPA2G180pp","PLIPA2G181d11pp","PSD120","PSD140","PSD141","PSD160","PSD161","PSD180","PSD181","PSSA120","PSSA140","PSSA141","PSSA160","PSSA161","PSSA180","PSSA181","DM_ACT","MRex","MRpp","DMPPDApp","DMPPDEex","EX_DMPPDA_e","EX_AP_e","EX_anl_e","Appp","ANLpp","13DAMPPabcpp","13DAMPPtex","15DAPabcpp","25DKGLCNt2rpp","25DKGLCNtex","26DAHtex","2DHGLCNkt_tpp","2DHGLCNtex","34DHCINMtex","34DHCINMtpp","34DHPHEtex","34DHPHEtpp","3MBt2pp","3MBtex","3OXOADPt_ex_","3OXOADPt_pp_","3_4DHBZ1t_pp_","3_4DHBZt2","4ABUTabcpp","4HBZtex","4HPRODCabcpp","4HPRODCtex","4HPROLTabcpp","4HPROLTtex","5APTNabcpp","5APTNtex","5OXPROabcpp","5OXPROtex","6ATHAtexi","6HNACt1pp","6HNACtex","ABUTt2pp","ABUTtex","ACACt2pp","ACACtex","ACALDtex","ACALDtpp","ACONCt4pp","ACONCtex","ACSERtex","ACSERtpp","ACTNtex","ACTNtpp","ACt2rpp","ACt4pp","ACt5pp","ACtex","ADEt2rpp","ADEtex","ADNtex","ADOCBLabcpp","ADOCBLtonex","AEP2abc","AEPtpp","AGM3Pt2pp","AGM4Pt2pp","AGMabcpp","AGMt2pp","AGMtex","AGtex","AGtpp","AKGt2rpp","AKGtex","ALAALAabcpp","ALAALAtex","ALAGLUabcpp","ALAHISabcpp","ALAHIStex","ALALEUabcpp","ALALEUtex","ALATHRabcpp","ALATHRtex","ALATRPabcpp","ALATRPtex","ALAabcpp","ALAt2pp","ALAt4pp","ALAtex","ALLTNt2rpp","ALLTNtex","APCt5","APCt6","APCtex","APCtpp","ARBt3ipp","ARGORNt7pp","ARGabcpp","ARGt2pp","ARGtex","ASNtex","ASO3tex","ASO4t2rpp","ASO4tex","ASPabcpp","ASPt2pp","ASPtex","BALAALAabcpp","BALAALAtex","BALABALAabcpp","BALABALAtex","BALAGLYabcpp","BALAGLYtex","BALALEUabcpp","BALALEUtex","BALAt2pp","BALAtex","BHBtpp","BILIVERDtex","BILIVERDtpp","BLAMDabcpp","BLAMDtex","BTDDtex","BTDDtpp","BTNt2ipp","BTNtex","BUTSO3abcpp","BUTSO3tex","BZt1pp","BZt2_pp","BZtex","CA2tex","CARNabcpp","CARNtex","CATECHOLtex","CATECHOLtpp","CAt6pp","CBItonex","CBIuabcpp","CBL1abcpp","CBL1tonex","CD2tex","CD2tpp","CELLex","CGLYabcpp","CGLYtex","CHLabcpp","CHLtex","CHOLS_ex_","CHOLSabc","CHOLt4ipp","CHORt2rpp","CHORtex","CINNMtex","CINNMtpp","CITt4pp","CITt_kt","CITtex","CLt3_2pp","CLtex","CMt2ex","CMt2ppi","CMtex","CMtexabc","CMtpp","CO2tex","CO2tpp","COBALT2tex","COBALT2tpp","CONFRLtex","CONFRLtpp","COtex","COtpp","CREATtex","CREATtpp","CRNabcpp","CRNtex","CRO2tex","CRO2tpp","CRO4abcpp","CRO4tex","CRTNtex","CRTNtpp","CSNt2pp","CSNtex","CU2tonex","CU2tpp","CUtex","CUtpp","CYANt2pp","CYANtex","CYSDabcpp","CYSDtex","CYSabcpp","CYStex","CYStpp","DALAabcpp","DALAt2pp","DALAtex","DAPabcpp","DAPtex","DARGabcpp","DARGtex","DASNabcpp","DASNtex","DCAtexi","DDCAtexi","DGUDBUTNabcpp","DGUDBUTNtex","DLEUabcpp","DLEUtex","DLYSabcpp","DLYStex","DMETtex","DMGLYBabcpp","DMGLYtex","DMSO2abcpp","DMSO2tex","DOPAtex","DOPAtpp","DORNabcpp","DORNtex","DPHEt2rpp","DPHEtex","DPROabcpp","DPROtex","DSERt2pp","DSERtex","DTYRt2rpp","DTYRtex","DVALabcpp","DVALtex","D_LACt2pp","D_LACtex","ECTOabcpp","ECTOtex","ETHAt2pp","ETHAtex","ETHSO3abcpp","ETHSO3tex","ETOHtex","ETOHtrpp","FALDep","FALDtex","FALDtpp","FALDtpp2","FBACabcpp","FE3DCITtonex","FERtex","FERtpp","FORt2pp","FORtex","FRUtex","FUMt2_2pp","FUMtex","G3PEabcpp","G3PGabcpp","GALCTt2rpp","GALCTtex","GALURt2rpp","GALURtex","GAt1","GAt2pp","GCALDtabcpp","GLCNt2rpp","GLCNtex","GLCRt2rpp","GLCRtex","GLCURt2rpp","GLCURtex","GLCabcpp","GLCtex","GLNtex","GLUTARt2rpp","GLUTARtex","GLUabcpp","GLUt2pp","GLUt4pp","GLUtex","GLYALDtex","GLYALDtpp","GLYBabcpp","GLYBt2pp","GLYBtex","GLYBti4pp","GLYCAt2rpp","GLYCAtex","GLYCLTt2rpp","GLYCLTt4pp","GLYCLTtex","GLYCOLtex","GLYCtex","GLYCtpp","GLYGLNabcpp","GLYGLNtex","GLYGLUabcpp","GLYGLUtex","GLYGLYabcpp","GLYGLYtex","GLYMETabcpp","GLYMETtex","GLYPHEabcpp","GLYPHEtex","GLYSERabcpp","GLYSERtex","GLYabc2pp","GLYt2pp","GLYt4pp","GLYtex","GTHRDabc2pp","GUAt2pp","GUAtex","GUDACabcpp","GUDACtex","GUDPTNabcpp","GUDPTNtex","H2O2tex","H2Otex","H2Otpp","H2St1pp","H2Stex","HAT40tex","HAT40tpp","HCYStabcpp","HCYStex","HDCAtexi","HDCEAtexi","HG2tex","HG2tppi","HGENTISt2pp","HGENTIStex","HISGLYabcpp","HISGLYtex","HISHISabcpp","HISHIStex","HISabcpp","HISt2rpp","HIStex","HOMt2pp","HOMtex","HOMtpp","HPAtexi","HXANt2pp","HXAtexi","HYXNtex","Htex","ICITt4pp","ICITt_kt","ICITtex","ILEabcpp","ILEt2rpp","ILEtex","IND3ACtex","IND3ACtpp","INDOLEt5","INDOLEt6","INDOLEtex","INDOLEtpp","INStex","ISETACabcpp","ISETACtex","Kabcpp","Kt2pp","Kt3pp","Ktex","LALALGLUtex","LCTSt3ipp","LEULEUabcpp","LEULEUtex","LEUabcpp","LEUt2rpp","LEUtex","LNLCtexi","LYSabcpp","LYStex","L_LACt2rpp","L_LACtex","MALt2_2pp","MALtex","MANabcpp","MANtex","MCSNtex","MCSNtpp","MELIBt3ipp","MEOHtex","MEOHtrpp","METD2abcpp","METabcpp","METtex","MG2tex","MG2tpp","MG2uabcpp","MMETt2pp","MMETtex","MNabcpp","MNtex","MOBDabcpp","MOBDtex","MSO3abcpp","MSO3tex","MXYLt5","MXYLt6","M_XYLtpp","M_Xylt1","NACt1pp","NACtex","NAt3_1p5pp","NAt3_1pp","NAt3_2pp","NAt3pp","NAtex","NCBMPTSabcpp","NCBMPTStex","NH4t3pp","NH4tex","NH4tpp","NI2tex","NI2tpp","NI2uabcpp","NO2t2rpp","NO2tex","NO3t2pp","NO3tabcpp","NO3tex","NONAtexi","NOtex","NOtpp","O2tex","O2tpp","OAAt2_2pp","OAAtex","OARGabcpp","OARGtex","OCDCAtexi","OCDCEAtexi","OCTAtexi","ORNabcpp","ORNtex","OROTt2_2pp","OROTtex","OXAtpp","OXFOtex","OXYLt5","OXYLt6","O_XYLtppi","O_Xylt1","PA120abcpp","PA160abcpp","PA161abcpp","PA180abcpp","PA181abcpp","PACALDtex","PACALtabcpp","PACt3","PACtex","PB2tex","PB2tpp","PE120abcpp","PE140abcpp","PE141abcpp","PE160abcpp","PE161abcpp","PE180abcpp","PE181abcpp","PEAL2tabc","PEALtabc","PEALtex","PEAMNtex","PEAtabcpp","PENTSOtex","PG120abcpp","PG140abcpp","PG141abcpp","PG160abcpp","PG161abcpp","PG180abcpp","PG181abcpp","PGP120abcpp","PGP140abcpp","PGP141abcpp","PGP160abcpp","PGP161abcpp","PGP180abcpp","PGP181abcpp","PHEDCAtexi","PHEHPAtexi","PHEHXAtexi","PHEME2abcpp","PHEMEtonex","PHENONAtexi","PHEOCTAtexi","PHEPTtexi","PHEt2rpp","PHEtex","PHPYRtex","PHPYRtpp","PIt2rpp","PIt7pp","PItex","PIuabcpp","PNTOt4pp","PNTOtex","PPAt4pp","PPAtex","PPItex","PPIuabcpp","PQQH2tpp","PQQtpp","PROGLYabcpp","PROGLYtex","PROabcpp","PROt2pp","PROt4pp","PROtex","PTAtex","PTRCabcpp","PTRCt2pp","PTRCtex","PTSLAtexi","PXYLt5","PXYLt6","PYDAMtex","PYDAMtpp","PYRt2rpp","PYRtex","P_XYLtppi","P_Xylt1","QUINtex","QUINtpp","RHA100tex","RHA100tpp","RHA101tex","RHA101tpp","RHA120tex","RHA120tpp","RHA121d6tex","RHA121d6tpp","RHA121tex","RHA121tpp","RHA140tex","RHA140tpp","RHA141d5tex","RHA141d5tpp","RHA141tex","RHA141tpp","RHA142tex","RHA142tpp","RHA160tex","RHA160tpp","RHA40tex","RHA40tpp","RHA50tex","RHA50tpp","RHA60tex","RHA60tpp","RHA70tex","RHA70tpp","RHA80tex","RHA80tpp","RHA90tex","RHA90tpp","RHAP100tex","RHAP100tpp","RHAP40tex","RHAP40tpp","RHAP50tex","RHAP50tpp","RHAP60tex","RHAP60tpp","RHAP70tex","RHAP70tpp","RHAP80tex","RHAP80tpp","RHAP90tex","RHAP90tpp","RHAT40tex","RHAT40tpp","RHAT60tex","RHAT60tpp","RIBabcpp","RIBtex","SBO3rt2pp","SBO3tex","SELabcpp","SELtex","SERt2rpp","SERt4pp","SERtex","SHEME2abcpp","SHEMEtonex","SKMt2pp","SKMtex","SO2tex","SO2tpp","SO3abcpp","SO3tex","SO4t2pp","SO4t4pp_1","SO4tex","SPMDabcpp","SPMDtex","SUCCt2_2pp","SUCCtex","SULFACabcpp","SULFACtex","SULabcpp","T4HCINNMtex","T4HCINNMtpp","TAURabcpp","TAURtex","TCYNTtex","TCYNTtpp","THRabcpp","THRt2pp","THRt4pp","THRtex","THYMt4pp","THYMtex","TNTtex","TNTtpp","TOLt5","TOLt6","TOLt7","TOLtex","TOLtppi","TRPt2rpp","TRPtex","TSULabcpp","TSULtex","TTDCAtexi","TTDCEAtexi","TTRCYCt2pp","TTRCYCt5","TTRCYCt7pp","TTRCYCtex","TTRCYCtpp","TUNGSabcpp","TUNGStex","TYRt2rpp","TYRtex","UHBZ1t_pp_","URAt2pp","URAtex","UREA2t2pp","UREAtabcpp","UREAtex","UREAtpp","URIt2rpp","URItex","VACCtex","VALabcpp","VALt2rpp","VALtex","VANKpp","VANLNtex","VANLTtex","VNLNpp","XANt2pp","XANtex","ZNabcpp","Zn2tex","pentso3abcpp", "4PCP","4PCPpp","A5PISO","ACGAMT","ACPS1","AGM3PA","AGM3PApp","AGM3PH","AGM4PA","AGM4PApp","AGM4PCP","AGM4PCPpp","AGM4PH","AGMH","AGMHE","ALAALAr","AM3PA","AM4PA","AM4PCP","ANHMK","BGLApp","CELASEpp","CELLS","COLIPAabcpp_kt","EDTXS1_kt","EDTXS2_kt","EDTXS5_kt","EDTXS6_kt","G1PACT","GF6PTA","GMHEPAT","GMHEPK","GMHEPPA","GTR1_kt","GTR2_kt","HEPCT","HEPK1_kt","HEPK2_kt","HEPK3_kt","HEPT1_kt","HEPT2_kt","KDOCT2","KDOPP","KDOPS","LIPAH","LPADSS2","LPSALAT","LPSGTR1_kt","LPSGTR2_kt","LPSGTR3_kt","LPSGTR4_kt","MCTP1App","MCTP1Bpp","MCTP2App","MDDCP1pp",
                   "MDDCP2pp","MDDCP3pp","MDDCP4pp","MDDCP5pp","MDDEP1pp","MDDEP2pp","MDDEP3pp","MDDEP4pp","MI1PP","MLDCP1App","MLDCP1Bpp","MLDCP2App","MLDCP2Bpp","MLDCP3App","MLDEP1pp","MLDEP2pp","MLTGY1pp","MLTGY2pp","MLTGY3pp","MLTGY4pp","MOAT2_kt","MOAT_kt","MPTG","MPTG2","OSKTabcpp","OSKTpp","PAPPT3","PETNT161_kt","PETNT181_kt","PGAMT","PLPSLpp","PLPSabctex","RHMNT1","RHMNT2","S7PI","TDSK2","U23GAAT2","UAAGDS","UACGAMDH2","UAG2E","UAGAAT2","UAGCVT","UAGDP","UAGPT3","UAMAGS","UAMAS","UAPGR","UDCPDP","UDCPDPpp","UDCPGT","UDCPPtppi","UDPG4E","UGMDDS","UHGADA2","UKQUIVORx","UKQUIVORy","UM3PL","UM4PCP","UM4PL","UQUIVOT","USHD1"]

In [11]:
## Find boundary reactions

boundary = model.boundary

In [12]:
## remove essential, blocked, and boundary reactions first

removereactions1 = list(essential) + blocked_reactions + boundary
removegenes = essentialgenes

for x in removereactions1:
    if x in model.reactions:
        model.remove_reactions([x])
    else:
        pass

print('%i reactions' % len(model.reactions))
print('%i genes' % len(model.genes))

1287 reactions
1490 genes


In [13]:
## remove transport and peripheral reactions that have not been removed

for x in combined:
    if x in model.reactions:
        model.remove_reactions([x])
    else:
        pass
    
print('%i reactions' % len(model.reactions))
print('%i genes' % len(model.genes))

912 reactions
1490 genes


In [14]:
## remove non-GPR reactions that have not been removed

for x in non_gar:
    if x in model.reactions:
        model.remove_reactions([x])
    else:
        pass
    
print('%i reactions' % len(model.reactions))
print('%i genes' % len(model.genes))

883 reactions
1490 genes


In [15]:
## remove essential genes
cobra.manipulation.remove_genes(model, essentialgenes)

print('%i reactions' % len(model.reactions))
print('%i genes' % len(model.genes))

778 reactions
1224 genes


In [16]:
cobra.io.write_sbml_model(model, "reduced_iJN1462c_mr.xml")
cobra.io.save_json_model(model, "reduced_iJN1462c_mr.json")